In [1]:
import datetime
import os
import webbrowser
import pyautogui as pg
import time
import glob
import shutil
import random
import pandas as pd
from sqlalchemy import create_engine
today = datetime.date.today()
download_path = r"C:\Users\810202558\Downloads"
baseDownloadedDir = r'D:\PROJECTS\Appportal\downloaded'

In [ ]:
data_spm = pd.DataFrame()
spm_files = glob.glob(r"downloaded\spm\*.csv")
for csv_file in spm_files:
    namefile = (csv_file.split("_"))[1]
    spm_temp = pd.read_csv(
        csv_file,
        dtype={
            "MASA PAJAK": "str",
            "TANGGAL BAYAR": "str",
            "NPWP": "str",
            "KPP": "str",
            "CABANG": "str",
        },
    )
    spm_temp["ADMIN"] = namefile
    data_spm = pd.concat([data_spm, spm_temp], axis=0, ignore_index=True)
data_spm["MASA PAJAK2"] = data_spm["MASA PAJAK"].str[2:3]
data_spm["MASA PAJAK"] = data_spm["MASA PAJAK"].str[:2]

In [ ]:
data_spm.loc[data_spm["MASA PAJAK"] == "n", "MASA PAJAK"]

In [ ]:
def generate_month_list():
    today = datetime.date.today()
    current_month = today.month
    months_list = [str(month).zfill(2) for month in range(1, current_month + 1)]
    return months_list

def create_folder(folder_name):
    try:
        os.mkdir(os.path.join('downloaded',folder_name))
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        pass

def delfiles(dir):
    try:
        for filename in os.listdir(dir):
            file_path = os.path.join(dir, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
    except Exception as e:
        print(f"Error: {e}")

def sorted_files(source_dir):
    csv_files = glob.glob(os.path.join(source_dir,'*.csv'))
    today_csv_files = []
    for csv_file in csv_files:
        stat = os.stat(csv_file)
        file_date = datetime.date.fromtimestamp(stat.st_ctime)
        if file_date == today:
            today_csv_files.append(csv_file)
    sorted_files = sorted(
        today_csv_files, key=os.path.getctime, reverse=False)
    return sorted_files

def moving_files(sorted_files,val):
    for filename in sorted_files:
        namefile = filename.split("\\")[-1]
        dest_dir = os.path.join(baseDownloadedDir,val, namefile)
        shutil.move(filename, dest_dir)

# MPN 

In [ ]:
baseUrl = 'https://appportal.intranet.pajak.go.id/portal/download/lsnfjkasbnfjnasjkfnjbnjnjknbkjnfjknbjkfnbkjfnbi3939489184.php?p1='
# kpp = ['001']
kpp = ['001','002','003','004','005','006','007','008','009','097']
tahun = '2023'
jns_pajak = '411100000'
tgl_awal = '01'
tgl_akhir = '31'
bulan = generate_month_list()
# bulan = ['01']
valuta = ['1','2','3']
# valuta = ['3']

In [ ]:
webbrowser.open('https://appportal.intranet.pajak.go.id/login/')
time.sleep(2)
pg.hotkey('tab')
pg.typewrite('810202558', interval=0.1)
pg.hotkey('tab')
pg.typewrite('Gengsu!sh3r3', interval=0.1)
pg.hotkey('enter')

time.sleep(3)

for val in valuta:
    mpn_dir = os.path.join(baseDownloadedDir,val)
    #bikin folder
    if not os.path.exists(mpn_dir):
         
         os.makedirs(mpn_dir)
    for adm in kpp:
      for bln in bulan:
               url = baseUrl+adm+tahun+jns_pajak+tgl_awal+tgl_akhir+bln+val
               webbrowser.open_new_tab(url)
               if val =='1':
                   time.sleep(random.randint(2,8))
               else:
                   time.sleep(random.randint(1,3))   
               

    sorted_files = sorted_files(download_path)
    num_files = len(kpp)*len(bulan) #per valuta
    while len(sorted_files)<num_files:
        time.sleep(10)
        sorted_files = sorted_files(download_path)
    
    delfiles(mpn_dir)
    moving_files(sorted_files, val)

# MPN USD

In [ ]:
url_usd = f'https://appportal.intranet.pajak.go.id/portal/detilperekaman/exportkpp.php?xunit=110&xtahun=2023&xbulan1=01&xbulan2={bulan[-1]}'
webbrowser.open_new_tab(url_usd)
time.sleep(3)
shutil.move(os.path.join(download_path,'detildollar.xls'), r'D:\PROJECTS\Appportal\downloaded\2\detildollar.xls')

# SPM

In [ ]:
baseUrl = 'https://appportal.intranet.pajak.go.id/portal/spm/dataspmcsv.php?'
# kpp = ['001']
kpp = ['001','002','003','004','005','006','007','008','009','097']
tahun = '2023'
tgl_awal = '01'
tgl_akhir = '31'
bulan = generate_month_list()
# bulan = ['01']

In [ ]:
ant = '&'
for adm in kpp:
    for bln in bulan:
        url = baseUrl+'p1='+tgl_awal+ant+'p2='+tgl_akhir+ant+'p3='+bln+ant+'p4='+tahun+ant+'p5='+adm
        webbrowser.open_new_tab(url)   
        time.sleep(random.randint(1,3))
    
    spm_dir = os.path.join('D:\PROJECTS\Appportal\downloaded','spm')
    #bikin folder
    if not os.path.exists(spm_dir):
         os.makedirs(spm_dir)

    sorted_files = sorted_files(download_path)
    num_files = len(kpp)*len(bulan) #per valuta
    while len(sorted_files)<num_files:
        time.sleep(10)
        sorted_files = sorted_files(download_path)
    
    delfiles(spm_dir)
    moving_files(sorted_files, 'spm')

# ETL

In [ ]:
idr_files = glob.glob(r'downloaded\1\*.csv')
usd_files = glob.glob(r'downloaded\2\*.csv')
dolar_file = glob.glob(r'downloaded\2\*.xls')
pbb_files = glob.glob(r'downloaded\3\*.csv')
spm_files = glob.glob(r'downloaded\spm\*.csv')

#### IDR

In [ ]:
data_mpn = pd.DataFrame()
for file in idr_files:
    namefile = (file.split("\\")[-1].split("_")[-1])[:3]
    temp = pd.read_csv(file, dtype={'NPWP': 'str', 'KPP': 'str', 'CAB': 'str','PTNTP':'str',
                                            'PTMSPJ': 'str', 'TGLBYR': 'str', 'BLNBYR': 'str', 'THNBYR': 'str'})
    temp['ADMIN'] = namefile
    data_mpn = pd.concat([data_mpn, temp], axis=0, ignore_index=True)

data_mpn.rename(columns={'THNBYR': 'TAHUNBAYAR',
                    'BLNBYR': 'BULANBAYAR', 'TGLBYR': 'TANGGALBAYAR'}, inplace=True)
data_mpn['DATEBAYAR'] = data_mpn['TAHUNBAYAR']+data_mpn['BULANBAYAR']+data_mpn['TANGGALBAYAR']
data_mpn['DATEBAYAR'] = pd.to_datetime(data_mpn['DATEBAYAR'],format='%Y%m%d')
data_mpn['MASA_PAJAK'] = data_mpn['PTMSPJ'].str[0:2]
data_mpn['MASA_PAJAK2'] = data_mpn['PTMSPJ'].str[2:4]
data_mpn['TAHUN_PAJAK'] = data_mpn['PTMSPJ'].str[4:]
# data_mpn['DATEBAYAR'] = pd.to_datetime(data_mpn[['TANGGALBAYAR', 'BULANBAYAR', 'TAHUNBAYAR']])

#### USD

In [ ]:
data_usd = pd.DataFrame()
for file in usd_files:
  namefile = (file.split("\\")[-1].split("_")[-1])[:3]
  temp = pd.read_csv(file, dtype={'NPWP': 'str', 'KPP': 'str', 'CAB': 'str','PTNTP':'str',
                                    'PTMSPJ': 'str', 'TGLBYR': 'str', 'BLNBYR': 'str', 'THNBYR': 'str'})
  data_usd['ADMIN'] = namefile
  data_usd = pd.concat([data_usd, temp], axis=0, ignore_index=True)
data_usd['NPWP'] = data_usd['NPWP'].str.strip()
data_usd['KPP'] = data_usd['KPP'].str.strip()
data_usd['CAB'] = data_usd['CAB'].str.strip()
data_usd.rename(columns={'THNBYR': 'TAHUNBAYAR',
                    'BLNBYR': 'BULANBAYAR', 'TGLBYR': 'TANGGALBAYAR'}, inplace=True)
data_usd['MASA_PAJAK'] = data_usd['PTMSPJ'].str[0:2]
data_usd['MASA_PAJAK2'] = data_usd['PTMSPJ'].str[2:4]
data_usd['TAHUN_PAJAK'] = data_usd['PTMSPJ'].str[4:]
data_usd['DATEBAYAR'] = data_usd['TAHUNBAYAR']+data_usd['BULANBAYAR']+data_usd['TANGGALBAYAR']
data_usd['DATEBAYAR'] = pd.to_datetime(data_usd['DATEBAYAR'],format='%Y%m%d')

In [ ]:
data_dollar = pd.read_excel(r'downloaded\2\detildollar.xls', engine='xlrd', usecols=['NTPN','JUMLAH RUPIAH'])

In [ ]:
data_usd = data_usd.merge(data_dollar,left_on='PTNTP', right_on='NTPN', how='left')

In [ ]:
data_usd = data_usd.drop(columns=['JUMLAH','NTPN'])
data_usd = data_usd.rename(columns={'JUMLAH RUPIAH':'JUMLAH'})

In [ ]:
data_mpnall = pd.concat([data_mpn,data_usd],axis=0,ignore_index=False)

In [ ]:
data_mpnall[['NOP','KET']] = data_mpnall[['NOP','KET']].fillna('-')

In [ ]:
data_mpnall.columns = [x.lower() for x in data_mpnall.columns]

In [ ]:
data_mpnall = data_mpnall.rename(columns={'cab':'cabang','kjs':'kdbyar','masa_pajak':'masa','tahun_pajak':'tahun','jumlah':'nominal','ptntp':'ntpn',
 'noskssp':'nosk','id':'billing','pembuat billing':'pembuat', 'masa_pajak2':'masa2'})
data_mpnall['nospm'] = ''
data_mpnall['source'] = 1
data_mpnall['extra'] = ''
data_mpnall['tipe'] = ''

In [ ]:
data_mpnall = data_mpnall.rename(columns={'kdbyar':'kdbayar'})

In [ ]:
data_mpnall = data_mpnall.drop(columns=['alamat','kd bank','kpp_penerima'])
data_mpnall = data_mpnall[['admin', 'npwp', 'kpp', 'cabang', 'nama', 'kdmap', 'kdbayar', 'masa',
       'tahun', 'tanggalbayar', 'bulanbayar', 'tahunbayar', 'datebayar',
       'nominal', 'ntpn', 'bank', 'nosk', 'nospm', 'tipe', 'source', 'extra',
       'billing', 'nop', 'pembuat', 'ket', 'masa2']]

#### SPM

In [ ]:
kpp = ['001','002','003','004','005','006','007','008','009','097']
spm_files = 'downloaded\spm'
spm_files = sorted_files(spm_files)

In [ ]:
spm_dir = os.path.join('D:\PROJECTS\Appportal\downloaded','spm')


In [ ]:
csv_files = glob.glob(os.path.join(download_path,'*.csv'))
today_csv_files = []
for csv_file in csv_files:
    stat = os.stat(csv_file)
    file_date = datetime.date.fromtimestamp(stat.st_ctime)
    today_csv_files.append(csv_file)
sorted_files = sorted(
    today_csv_files, key=os.path.getctime, reverse=False)

In [ ]:
kpp = ["001", "002", "003", "004", "005", "006", "007", "008", "009", "097"]
tahun = "2023"
jns_pajak = "411100000"
tgl_awal = "01"
tgl_akhir = "31"
bulan = generate_month_list()
# bulan = ['01']
valuta = ["1", "2", "3"]
spm_sorted = sorted_files(download_path)
num_files = len(kpp) * len(bulan)  # per va luta
while len(spm_sorted) < num_files:
    time.sleep(10)
    spm_sorted = sorted_files(download_path)


In [ ]:
spm_downloaded = glob.glob(os.path.join(download_path, "Data_SPM*.csv"))

In [ ]:
csv_files = glob.glob(os.path.join(download_path,'*.csv'))

In [ ]:
x = csv_files[0].split("\\")
y = '\\'.join(a for a in x[:-1])

In [ ]:
spm = pd.read_csv('downloaded\spm\SPM_001_01.csv',dtype={
                "MASA PAJAK": "str",
                "TANGGAL BAYAR": "str",
                "NPWP": "str",
                "KPP": "str",
                "CABANG": "str",
                "NTPN": "str",
            },)

In [ ]:
spmkp_files = glob.glob(r"downloaded\spmkp\*.csv")
data_spmkp = pd.DataFrame()
for file in spmkp_files:
  namefile = namefile = (file.split("_"))[1]
  spmkp_temp = pd.read_csv(file, parse_dates=['TGL SPMKP'], dtype={'NPWP': 'str', 'KD MAP': 'str','NO SPMKP': 'str'})
  spmkp_temp['ADMIN'] = namefile
  data_spmkp = pd.concat([data_spmkp, spmkp_temp],axis=0, ignore_index=True)
data_spmkp['ADMIN'] = data_spmkp['ADMIN'].str.replace('.csv','')
data_spmkp.rename(columns={'TGL SPMKP':'TANGGAL','KD MAP':'KDMAP'},inplace=True)
data_spmkp['KPP'] = data_spmkp['NPWP'].str[10:13]
data_spmkp['CABANG'] = data_spmkp['NPWP'].str[14:]
data_spmkp['NPWP'] = data_spmkp['NPWP'].str[:9]
data_spmkp['BULAN'] = data_spmkp['TANGGAL'].dt.month
data_spmkp['TAHUN'] = data_spmkp['TANGGAL'].dt.year
data_spmkp['NOMINAL'] = data_spmkp['NILAI SPMKP'].str.replace(',', '').astype('int64')
data_spmkp.columns = [x.lower() for x in data_spmkp.columns]
spmkp_columns = ['admin','npwp','kpp','cabang','kdmap','bulan','tahun','tanggal','nominal']
data_spmkp = data_spmkp[spmkp_columns]

In [ ]:
pbk_files = glob.glob(r"downloaded\pbk\*.csv")
data_pbk = pd.DataFrame()
for file in pbk_files:
  namefile = namefile = (file.split("_"))[1]
  pbk_temp = pd.read_csv(file, parse_dates=['TGL_DOKUMEN','TGL_BERLAKU'],
                            dtype= {'NPWP':'str', 'KPP':'str', 'CAB':'str','KPPADM_LB':'str', 'NPWP.1':'str','KPP.1':'str',
                                    'CAB.1':'str','MAP.1':'str','KD SETOR.1':'str','MAP':'str','MAP.1':'str','KD SETOR':'str',
                                    'KPPADM_KB':'str'})
  pbk_temp['ADMIN'] = namefile
  data_pbk = pd.concat([data_pbk, pbk_temp], axis=0, ignore_index=True)
data_pbk['JUMLAH_PBK']= data_pbk['JUMLAH_PBK'].str.replace(',','')
data_pbk['JUMLAH_PBK'] = data_pbk['JUMLAH_PBK'].astype('int64')
data_pbk['TGL_DOKUMEN'] = pd.to_datetime(data_pbk['TGL_DOKUMEN'], format='%Y%m%d')
data_pbk['TAHUN'] = data_pbk['TGL_DOKUMEN'].dt.year

data_pbk.rename(columns={'NOMOR_PBK':'NOPBK','TGL_DOKUMEN':'TANGGALDOC','TGL_BERLAKU':'TANGGALBERLAKU',
                          'JUMLAH_PBK':'NOMINAL','CURRENCY_PBK':'CURRENCY','TIPE_PBK':'TIPE','FG_STATUS':'STATUS','NPWP':'NPWP','KPP':'KPP',
                          'CAB':'CABANG','NAMA':'NAMA','MAP':'KDMAP','KD SETOR':'KDBAYAR','MASA PAJAK':'MASA_PAJAK','TAHUN PAJAK':'TAHUN_PAJAK',
                          'KPPADM_LB':'KPP_ADMIN','NPWP.1':'NPWP2','KPP.1':'KPP2','CAB.1':'CABANG2','NAMA.1':'NAMA2','MAP.1':'KDMAP2',
                          'KD SETOR.1':'KDBAYAR2','MASA PAJAK.1':'MASA_PAJAK2','TAHUN PAJAK.1':'TAHUN_PAJAK2','KPPADM_KB':'KPP_ADMIN2',
                          'NTPN':'NTPN','NO PROD HUKUM':'NO_PROD_HUKUM'},inplace=True)

data_pbk = data_pbk.filter(['ADMIN','TAHUN','NOPBK','TANGGALDOC','TANGGALBERLAKU','NOMINAL','CURRENCY','TIPE','STATUS',
                            'NPWP','KPP','CABANG','NAMA','KDMAP','KDBAYAR','MASA_PAJAK','TAHUN_PAJAK','KPP_ADMIN','NPWP2','KPP2','CABANG2','NAMA2',
                            'KDMAP2','KDBAYAR2','MASA_PAJAK2','TAHUN_PAJAK2','KPP_ADMIN2','NTPN','NO_PROD_HUKUM'])



data_pbk['NO_PROD_HUKUM'].fillna('',inplace=True)
colmap = {'masa_pajak':'masapajak','masa_pajak2':'masapajak2', 'tahun_pajak':'tahunpajak','tahun_bayar':'tahunbayar','bulan_bayar':'bulanbayar'}
data_pbk.columns = [col.lower() for col in data_pbk.columns]
data_pbk.rename(columns=colmap, inplace=True)

data_pbk['tanggalbayar'] = data_pbk['tanggaldoc'].dt.day
data_pbk['bulanbayar'] = data_pbk['tanggaldoc'].dt.month
data_pbk['tahunbayar'] = data_pbk['tanggaldoc'].dt.year

In [ ]:
from mpnspm import etl_mpn

usd_files = glob.glob(r"downloaded\2\*.csv")
data_usd = etl_mpn(usd_files)

data_dollar = pd.read_excel(
    r"downloaded\2\detildollar.xls",
    engine="xlrd",
    usecols=["NTPN", "JUMLAH RUPIAH"]
)


In [ ]:
data_usd = data_usd.merge(data_dollar, left_on="PTNTP", right_on="NTPN", how="left")
data_usd = data_usd.drop(columns=["JUMLAH", "NTPN"])
data_usd = data_usd.rename(columns={"JUMLAH RUPIAH": "JUMLAH"})

In [ ]:
data_usd.pivot_table(index=['BULANBAYAR'], columns='ADMIN',aggfunc='sum' ).reset_index()

In [ ]:
data_usd['JUMLAH'].sum()

In [2]:
from sqlalchemy import create_engine
mysql_con = create_engine("mysql://root@localhost/dimensi")
data = pd.read_parquet("D:\DATA\dim_klu.parquet")
data.to_sql('dim_klu', con=mysql_con, index=False, if_exists='replace')

1807

In [ ]:
mpn = mpn[['admin','nominal']]
mpn.pivot_table(index='admin', aggfunc='sum').reset_index().to_excel(r'D:\HASIL KUERI\mpntojuli.xlsx')

In [4]:
from mpnspm import etl_mpnspm
idr_files = glob.glob(r"downloaded\1\*.csv")
usd_files = glob.glob(r"downloaded\2\*.csv")
dolar_file = glob.glob(r"downloaded\2\*.xls")
pbb_files = glob.glob(r"downloaded\3\*.csv")
spm_files = glob.glob(r"downloaded\spm\*.csv")
spmkp_files = glob.glob(r"downloaded\spmkp\*.csv")
pbk_files = glob.glob(r"downloaded\pbk\*.csv")

data_mpnspm = etl_mpnspm(idr_files, usd_files, dolar_file, pbb_files, spm_files)

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [21]:
from mpnspm import etl_mpn
usd_files = glob.glob(r"downloaded\2\*.csv")
dolar_file = glob.glob(r"downloaded\2\*.xls")

In [32]:
data_usd = etl_mpn(usd_files)
data_usd.columns

Index(['NPWP', 'KPP', 'CAB', 'NAMA', 'KDMAP', 'KJS', 'PTNTP', 'PTMSPJ',
       'TAHUNBAYAR', 'BULANBAYAR', 'TANGGALBAYAR', 'JUMLAH', 'BANK', 'NOSKSSP',
       'ID', 'NOP', 'PEMBUAT BILLING', 'KET', 'ADMIN', 'DATEBAYAR',
       'MASA_PAJAK', 'MASA_PAJAK2', 'TAHUN_PAJAK'],
      dtype='object')

In [79]:
data_dollar = pd.read_excel(
    r"downloaded\2\detildollar.xls",
    engine="xlrd", dtype={'MASA':'str','NPWP':'str','KPP':'str','CAB':'str'}
)

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [82]:
data_dollar['JUMLAH RUPIAH'].sum()

2602309763796.7803

In [71]:

masterfile_con = create_engine("mysql://root@localhost/registrasi".format("sgwi2341"))
data_dollar = pd.read_excel(
    r"downloaded\2\detildollar.xls",
    engine="xlrd", dtype={'MASA':'str','NPWP':'str','KPP':'str','CAB':'str'}
)
data_dollar['TANGGAL SETOR'] = pd.to_datetime(data_dollar['TANGGAL SETOR'], dayfirst=True)
data_dollar['TAHUNBAYAR'] = data_dollar["TANGGAL SETOR"].dt.year
data_dollar['BULANBAYAR'] = data_dollar["TANGGAL SETOR"].dt.month
data_dollar['TANGGALBAYAR'] = data_dollar["TANGGAL SETOR"].dt.day
data_dollar['MASA_PAJAK'] = data_dollar['MASA'].str[:2]
data_dollar['MASA_PAJAK2'] = data_dollar['MASA'].str[2:4]
data_dollar['TAHUN_PAJAK'] = data_dollar['MASA'].str[4:]

dollar_rename = {
    "NAMA WAJIB PAJAK":'NAMA',"MAP":"KDMAP","KD.SETOR":'KJS',"JUMLAH BAYAR":'JUMLAH',"MASA":"PTMSPJ","NTPN":"PTNTP","TANGGAL SETOR":"DATEBAYAR",
    "NO KETETAPAN":"NOSKSSP"
}
data_dollar.rename(columns=dollar_rename, inplace=True)
data_dollar['NPWP'] =  data_dollar['NPWP'].str.strip()
data_dollar['NPWP15'] = data_dollar['NPWP']+data_dollar['KPP']+data_dollar['CAB']
data_dollar['BANK'] = ''
data_dollar['ID'] = ''
data_dollar['NOP'] = ''
data_dollar['PEMBUAT BILLING'] = ''
data_dollar['KET'] = ''
filter_npwp = ",".join([f"'{x.strip()}'" for x in data_dollar['NPWP15'].unique()])
# data_dollar = data_dollar[data_mpn.columns]

mf_kueri = f'''select KPPADM,NPWP15 from registrasi.sidjp_masterfile where NPWP15 in({filter_npwp})'''
mf = pd.read_sql(mf_kueri, con=masterfile_con)
data_dollar = data_dollar.merge(mf, on='NPWP15', how='left')
data_dollar.rename(columns={'KPPADM':'ADMIN'}, inplace=True)
data_dollar.drop(columns=['NPWP15'], inplace=True)

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [72]:
data_dollar.head()

,NO,NPWP,KPP,CAB,NAMA,KDMAP,KJS,JUMLAH,PTMSPJ,DATEBAYAR,...,TANGGALBAYAR,MASA_PAJAK,MASA_PAJAK2,TAHUN_PAJAK,BANK,ID,NOP,PEMBUAT BILLING,KET,ADMIN
0,1,029066883,006,000,IPPOSHA INDONESIA,411126,100,8512.42,12122022,2023-01-10,...,10,12,12,2022,,,,,,097
1,2,029066883,006,000,IPPOSHA INDONESIA,411126,100,8512.42,01012023,2023-02-13,...,13,01,01,2023,,,,,,097
2,3,029066883,006,000,IPPOSHA INDONESIA,411126,100,8512.42,02022023,2023-03-10,...,10,02,02,2023,,,,,,097
3,4,029066883,006,000,IPPOSHA INDONESIA,411126,100,8512.42,03032023,2023-04-10,...,10,03,03,2023,,,,,,097
4,5,029066883,006,000,IPPOSHA INDONESIA,411126,100,4158.00,04042023,2023-05-10,...,10,04,04,2023,,,,,,097


In [73]:
data_dollar['JUMLAH'].sum()

174220021.37

In [18]:
data_dollar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   NO               29 non-null     int64         
 1   NPWP             29 non-null     object        
 2   KPP              29 non-null     object        
 3   CAB              29 non-null     object        
 4   NAMA             29 non-null     object        
 5   KDMAP            29 non-null     int64         
 6   KJS              29 non-null     int64         
 7   JUMLAH           29 non-null     float64       
 8   PTMSPJ           29 non-null     object        
 9   DATEBAYAR        29 non-null     datetime64[ns]
 10  TANGGAL REKAM    0 non-null      float64       
 11  TANGGAL KURS     29 non-null     int64         
 12  KURS TENGAH BI   29 non-null     int64         
 13  JUMLAH RUPIAH    29 non-null     float64       
 14  SUMBER           29 non-null     object     

In [14]:
data_dollar['PTMSPJ']

0     12122022
1      1012023
2      2022023
3      3032023
4      4042023
5      5052023
6      6062023
7      4042023
8      5052023
9      6062023
10     1122022
11    12122022
12     1012023
13     2022023
14     3032023
15     4042023
16     5052023
17     6062023
18        2022
19     1122018
20    12122022
21     1012023
22     2022023
23     3032023
24     4042023
25     5052023
26     6062023
27     1122022
28     1122022
Name: PTMSPJ, dtype: int64